In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf


In [18]:
#download from here https://github.com/le-scientifique/torchDatasets/raw/master/dbpedia_csv.tar.gzdbpedia = pd.read_csv('data/dbpedia_csv/train.csv', header=None)
train_X = dbpedia[2]
train_y = dbpedia[0]
dbpedia = pd.read_csv('data/dbpedia_csv/test.csv', header=None)
test_X = dbpedia[2]
test_y = dbpedia[0]

In [24]:
train_len = train_X.apply(lambda x: len(x.split(' '))).get_values()
test_len = test_X.apply(lambda x: len(x.split(' '))).get_values()

In [25]:
vocab_processor=tf.contrib.learn.preprocessing.VocabularyProcessor(50)

In [26]:
train_X_t = vocab_processor.fit_transform(train_X)
test_X_t = vocab_processor.transform(test_X)

In [27]:
train_X = np.array(list(train_X_t))
test_X = np.array(list(test_X_t))

In [28]:
def batcher(num_samples,size):
    idx = np.random.choice(np.arange(num_samples), batch_size)
    return idx

In [35]:
batch_size=128
vocabulary_size = len(vocab_processor.vocabulary_)
embed_size=100
n_classes=15
graph = tf.Graph()
with graph.as_default():
    raw_y=tf.placeholder(tf.int64, shape=[batch_size])
    tf_y = tf.one_hot(raw_y,15, dtype=tf.int64)
    tf_X = tf.placeholder(tf.int64, shape=[batch_size, 50])
    tf_lens = tf.placeholder(tf.int64, shape=[batch_size])
    
    embeddings = tf.Variable(tf.random_uniform([vocabulary_size,embed_size],-1.0,1.0)) 
    t_embed = tf.nn.embedding_lookup(embeddings, tf_X) #shape [batch_size, seq_length, embedding_size]
    s_embed = tf.unstack(t_embed,axis=1) # list of [batch_size, embedding_size]
    cell = tf.contrib.rnn.LSTMCell(num_units=embed_size)
    #outputs,state = tf.contrib.rnn.static_rnn(cell, s_embed, dtype=tf.float32)
    outputs, state = tf.nn.dynamic_rnn(cell, t_embed, sequence_length=tf_lens, dtype=tf.float32)
    
    weights = tf.Variable(tf.truncated_normal([embed_size,n_classes]))
    bias = tf.Variable(tf.random_uniform([n_classes], -1.0,1.0))
    logits = tf.add(tf.matmul(state.h, weights), bias)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_y, logits=logits))
    opt = tf.train.AdamOptimizer(0.005).minimize(loss)
    
    predictions=tf.argmax(tf.nn.softmax(logits),1)
    correct_prediction = tf.equal(raw_y, predictions)
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    init_op = tf.global_variables_initializer()


In [37]:

n_steps=500
with tf.Session(graph=graph) as sess:
    sess.run(init_op)
    average_loss=0.0
    for step in range(n_steps):
        idx=batcher(train_X.shape[0],batch_size )
        feed_dict= {tf_X:train_X[idx], raw_y:train_y[idx], tf_lens:train_len[idx]}
        l,_=sess.run([loss, opt], feed_dict=feed_dict)
        average_loss +=l
        if (step>0) and (step%100==0):
            
            idx=batcher(test_X.shape[0],batch_size*10)
            test_dict = {tf_X:test_X[idx], raw_y:test_y[idx], tf_lens:test_len[idx]}
            print("average loss:", l/100)
            print("accuracy",accuracy.eval(feed_dict=test_dict))
            print(predictions.eval(feed_dict=test_dict))
           
            print(test_y[idx].get_values())
            print('-'*100)
            average_loss=0.0
            

average loss: 0.00571637988091
accuracy 0.820313
[ 2 10  6  9  5  1  5 11 11  1  6 12  7  7  5  6 11 12 10  6  8  6  8  3 12
 14 13  6  2  6  4 10  7  3 10  6  5  6 10  5  6  9  6  9  5 14  5 11  5  7
 12  8  7  6  2 11 11 13  4  5 13  8  2  4 12  6  4  5 11  4 12 13  7 11 12
  9  4  6  7  4  9 12  2 14  6 10  6  4  2  1  9 13 14 10 11  6 10 14  6  8
  4 13 13 13  9  3 10  4 11 10  4 12  6 10 13  1  5  3 12  1  1  8  3  4  5
  5  2  9]
[ 2 11  6  9  3  1  3 11 11  1  6 12  7  1  5  6 11 12 11  6  8  9  8  3 12
 14 13  6  2  6  4 10  2  3 10  6  5  6 10  5  7  9  1  8  5 14 11 11  1  7
 12  8  7  6  2 11 11 13  3  5 13  8  2  4 12  6  4  5 11  4 12 13  7 11 12
  9  4  6  7  4  9 12  2 14  6 10  1  4  2 10  9 13 13 11 11  6 11 14  6  8
  4 13 13 13  9  3 10  3 11 10  4 12  6 10 13  1  5 11 12  8  7  8  3  4  1
  5  2  9]
----------------------------------------------------------------------------------------------------
average loss: 0.00251570433378
accuracy 0.898438
[ 5  5 13 14  3 13 